In [1]:
import xarray as xr

In [3]:
gfs_file = 'model_data/gfs.t06z.pgrb2.0p25.f000'

gfs_ds = xr.open_dataset(gfs_file, engine="cfgrib",backend_kwargs={'filter_by_keys': {'typeOfLevel':'isobaricInhPa'}})


skipping variable: paramId==228164 shortName='tcc'
Traceback (most recent call last):
  File "/Applications/anaconda3/envs/flc_local/lib/python3.12/site-packages/cfgrib/dataset.py", line 701, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/Applications/anaconda3/envs/flc_local/lib/python3.12/site-packages/cfgrib/dataset.py", line 629, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='isobaricInhPa' value=Variable(dimensions=('isobaricInhPa',), data=array([1000.,  975.,  950.,  925.,  900.,  850.,  800.,  750.,  700.,
        650.,  600.,  550.,  500.,  450.,  400.,  350.,  300.,  250.,
        200.,  150.,  100.,   70.,   50.,   40.,   30.,   20.,   15.,
         10.,    7.,    5.,    3.,    2.,    1.])) new_value=Variable(dimensions=('isobaricInhPa',), data=array([1000.,  975.,  950.,  925.,  900.,  850.,  800.,  750.,  700.,
        650.,  600.,  550.,  500.,  450.,  400.,  350.,  300.,

In [ ]:
gfs_ds